# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import time
from PIL import Image
import  random
import glob
import numpy as np

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = transforms.Compose([transforms.Resize(224),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

#data_transforms_2 = transforms.Compose([transforms.Resize(224),
#                                 transforms.CenterCrop(224),
#                                 transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0),
#                                 transforms.RandomHorizontalFlip(p=0.5)])

# TODO: Load the datasets with ImageFolder
train_image_datasets = datasets.ImageFolder(data_dir + '/train', transform=data_transforms)
test_image_datasets = datasets.ImageFolder(data_dir + '/test', transform=data_transforms)
valid_image_datasets = datasets.ImageFolder(data_dir + '/valid', transform=data_transforms)
# TODO: Using the image datasets and the trainforms, define the dataloaders
train_dataloaders =  torch.utils.data.DataLoader(train_image_datasets, batch_size=64, shuffle=True)
test_dataloaders =  torch.utils.data.DataLoader(test_image_datasets, batch_size=64, shuffle=True)
valid_dataloaders =  torch.utils.data.DataLoader(valid_image_datasets, batch_size=64, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
#cat_to_name

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [5]:
# TODO: Build and train your network
model = models.vgg11(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /root/.torch/models/vgg11-bbd30ac9.pth
100%|██████████| 531456000/531456000 [00:09<00:00, 53867723.22it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [6]:
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('drop', nn.Dropout(p=.5)),
                          ('fc2', nn.Linear(4096, 1024)),
                          ('relu', nn.ReLU()),
                          ('drop2', nn.Dropout(p=.5)),
                          ('fc3', nn.Linear(1024, 102)),
                          ('relu', nn.ReLU()),
                          ('drop2', nn.Dropout(p=.5)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [7]:
# Implement a function for the validation pass
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in testloader:
        images,labels=images.to(device),labels.to(device)
        #images.resize_(images.shape[0], 784)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [8]:
# TODO: Train a model with a pre-trained network
device='cuda'
epochs = 3
model.train()
for e in range(epochs):
    steps=0
    print_every = 10
    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)
    #train_dataloaders
    running_loss=0
    for ii, (inputs, labels) in enumerate(train_dataloaders):
        
        steps += 1
        #inputs.resize_(inputs.size()[0], 50176)
        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()
        optimizer.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if steps % print_every == 0:
             model.eval()
             with torch.no_grad():
                    model.to(device)
                    test_loss, accuracy = validation(model, test_dataloaders, criterion)
             print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(test_dataloaders)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(test_dataloaders)))
             running_loss=0
             model.train()
        
print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Epoch: 1/3..  Training Loss: 7.546..  Test Loss: 3.971..  Test Accuracy: 0.177
Epoch: 1/3..  Training Loss: 3.722..  Test Loss: 2.969..  Test Accuracy: 0.328
Epoch: 1/3..  Training Loss: 2.818..  Test Loss: 2.230..  Test Accuracy: 0.488
Epoch: 1/3..  Training Loss: 2.500..  Test Loss: 1.800..  Test Accuracy: 0.574
Epoch: 1/3..  Training Loss: 1.917..  Test Loss: 1.496..  Test Accuracy: 0.630
Epoch: 1/3..  Training Loss: 1.661..  Test Loss: 1.278..  Test Accuracy: 0.677
Epoch: 1/3..  Training Loss: 1.765..  Test Loss: 1.142..  Test Accuracy: 0.696
Epoch: 1/3..  Training Loss: 1.485..  Test Loss: 1.079..  Test Accuracy: 0.700
Epoch: 1/3..  Training Loss: 1.564..  Test Loss: 0.956..  Test Accuracy: 0.758
Epoch: 1/3..  Training Loss: 1.376..  Test Loss: 0.873..  Test Accuracy: 0.771
Epoch: 2/3..  Training Loss: 0.974..  Test Loss: 1.043..  Test Accuracy: 0.733
Epoch: 2/3..  Training Loss: 0.833..  Test Loss: 0.933..  Test Accuracy: 0.760
Epoch: 2/3..  Training Loss: 0.947..  Test Loss: 0.9

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [9]:
# TODO: Do validation on the test set



## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [8]:
model.class_to_idx = train_image_datasets.class_to_idx
torch.save(model.state_dict(), 'checkpoint.pth')


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [8]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())
model.load_state_dict(state_dict)

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.11.weight', 'features.11.bias', 'features.13.weight', 'features.13.bias', 'features.16.weight', 'features.16.bias', 'features.18.weight', 'features.18.bias', 'classifier.fc1.weight', 'classifier.fc1.bias', 'classifier.fc2.weight', 'classifier.fc2.bias', 'classifier.fc3.weight', 'classifier.fc3.bias'])


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [23]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    im=Image.open(image)
    print(im.size)
    w,h = im.size
    ar=w/h
    if w>h:
        new_h=256
        new_w=245*ar
    else:
        new_w=256
        new_h=256/ar
        
    print(w,h,new_w,new_h)
    l=(new_w-224)/2
    r=224+l
    t=(new_h-224)/2
    b=t+224
    print(l,r,t,b)
    im_crop = im.crop((l,r,t,b))
    print(im_crop.size)
    np_image = np.array(im_crop)
    np_image=np_image/255
    #np_image = (np.array(im_crop))/255
    
    
    #imshow(im)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    np_image = (np_image - mean) / std
    np_image = np_image.transpose((2, 0, 1))

    processed_image = torch.from_numpy(np_image)

    tensor_image = processed_image.float()

    #print(type(tensor_image))
    return tensor_image
    # TODO: Process a PIL image for use in a PyTorch model
   
#img = random.choice(glob.glob('flowers/test/*/*.jpg'))
#imshow(process_image(img))
#print(img)


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [9]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [18]:
def predict(image_path, ckp_path, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
   
    state_dict = torch.load(ckp_path)
    model.load_state_dict(state_dict)
    model.to('cuda')
    #print(image_path)
    #img = process_image(image_path)
    
    # TODO: Implement the code to predict the class from an image file
    with torch.no_grad():
       #img = img.to('cuda')
       model.eval()
       output = model.forward(image_path)
       probabilities=torch.exp(output)
       p=probabilities.topk(topk)
       print(p)
       return probabilities
#img = random.choice(glob.glob('flowers/valid/*/*.jpg'))
for ii, (inputs, labels) in enumerate(valid_dataloaders):
   inputs, labels = inputs.to('cuda'), labels.to('cuda')
   #top_probs, top_labels, top_flowers = 
   predict(inputs,'checkpoint.pth')
   break
                                                 

(tensor(1.00000e-02 *
       [[ 1.3108,  1.1743,  1.1665,  1.1384,  1.1297],
        [ 1.2304,  1.2148,  1.1967,  1.1615,  1.1556],
        [ 1.2539,  1.2028,  1.2025,  1.1967,  1.1958],
        [ 1.3413,  1.2994,  1.2989,  1.2381,  1.2317],
        [ 1.3440,  1.2932,  1.2699,  1.2490,  1.2291],
        [ 1.3431,  1.3288,  1.2592,  1.2392,  1.1995],
        [ 1.2127,  1.1386,  1.1202,  1.1172,  1.1170],
        [ 1.2304,  1.2220,  1.2116,  1.1764,  1.1627],
        [ 1.3696,  1.3695,  1.3295,  1.2782,  1.2205],
        [ 1.3249,  1.3161,  1.2642,  1.2147,  1.2140],
        [ 1.3652,  1.3442,  1.3126,  1.2577,  1.2372],
        [ 1.4595,  1.4579,  1.4027,  1.3794,  1.3767],
        [ 1.3874,  1.3223,  1.2942,  1.2629,  1.2611],
        [ 1.1875,  1.1574,  1.1541,  1.1084,  1.1043],
        [ 1.3170,  1.2348,  1.2197,  1.1477,  1.1386],
        [ 1.3087,  1.2958,  1.2461,  1.2390,  1.1965],
        [ 1.4929,  1.3037,  1.2962,  1.2945,  1.2754],
        [ 1.3515,  1.2963,  1.2888,  1.2582

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [15]:
# TODO: Display an image along with the top 5 classes